# Illustrator multi-agent

This is an exercise to complete in home


Given raw text our autonomous agent will:

- **Search a directory and load articles.**
- Read the article and analyze its topic and structure.
- Decide how many images suit better to the article. Focusing on improving attractiveness while not harming readability.
- Decide where in the article new companion images should be located.
- Create detailed image descriptions suitable to create suggested images using an external image generator model.
- **Call the image generator to create the images.**
- **Download the images and store them in disk**

In [ ]:
import os
import json

from crewai import LLM, Agent, Task, Crew, Process
from crewai.tools import tool
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool, DallETool)
from dotenv import load_dotenv

from IPython.display import Image

# Configure LLMs and providers

In [ ]:
SELECTED_MODEL="sonar-pro"
URL_PROVIDER="https://api.perplexity.ai/"

In [ ]:
load_dotenv(override=True)

In [ ]:
API_KEY = os.getenv("API_KEY")

In [ ]:
def get_llm():
    return LLM(model=SELECTED_MODEL, 
               api_key=API_KEY, 
               base_url=URL_PROVIDER)

In [ ]:
llm = get_llm()

# General structure of a Crew AI agent

1. **Tools**
2. Agents
3. Tasks
4. Crew

## 1. Tools

In [ ]:
docs_tool = DirectoryReadTool(directory='./shared_community_manager_docs',
                              file_types=['.txt', '.md'],)
file_tool = FileReadTool()

How to create your own tools?

How to create them with enough quality?

**Docstrings and type hints!!**

In [ ]:
@tool
def generate_image(description: str) -> str:
    """
    Generate an image based on the provided description using Nano Banana
    and return the URL of the generated image.

    Parameters:
        description (str): A detailed description of the image to be generated.

    Returns:
        str: URL of the generated image.
    """

    # Add your code here
    return url

In [ ]:
@tool
def store_url_image_in_disk(url: str) -> str:
    """
    Store the image from the provided URL in disk and return the local path.

    Parameters:
        url (str): URL of the image to be stored.

    Returns:
        str: Local path where the image is stored.
    """

    # Add your code here
    return local_path

**Tip**: Check [Crew AI's tools catalog](https://docs.crewai.com/en/concepts/tools#available-crewai-tools). There are lots of useful tools ready to use.

## 1. Agents

Add the new tools to the agent

In [ ]:
image_creator_agent = Agent(
    role="Senior image creator",
    goal="To generate high-quality image descriptions that align with the content of articles.",
    backstory="You have been analyzing articles for years and know how to create compelling images that capture the essence of the content and increase reader engagement. "
                "You are skilled in understanding the nuances of different topics and can generate image descriptions that are both relevant and engaging.",
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm
    )

## 2. Tasks

Don't forget to adapt the tasks to use the tools

In [ ]:
image_description_task = Task(
    name="Image description generation",
    description="You will generate good quality image descriptions for an article. "
                "Depending on the topic and the length of the article you will decide the number of images. Always between 1 and 3. "
                "Read the article from the shared directory.",
    expected_output="""A list containing JSON objects, each one with 3 keys: 
                    'order' which contains an integer that indicates the order (1,2 ..) of the generated image. 
                    'previous_text': last 4 or 5 words where you decide to insert the image below, just for position reference. 
                    'description': a very detailed image description that can be used to create the image.""",
    context=[],
    agent=image_creator_agent
)

image_creation_taks = Task(
    name="Image creation",
    description="You will create images based on the provided descriptions. "
                "Use the description to generate an image that accurately reflects the content and context of the description. "
                "Ensure that the images are of high quality and resolution, suitable for use in articles. "
    expected_output="A list of URLs pointing to the generated images, each corresponding to the provided descriptions.",
    context=[image_description_task],
    agent=image_creator_agent
)

## 3. Crew

In [ ]:
crew = Crew(agents=[image_description_task],
            tasks=[image_description_task, image_creation_task],
            process=Process.sequential,
            verbose=True)

## Launch execution

In [ ]:
result = crew.kickoff()

## Results

In [ ]:
print(result.raw)

## Costs

In [ ]:
crew.usage_metrics

# Let's visualize the results

https://aistudio.google.com

Model: Nano Banana

In [ ]:
result_json = json.loads(result.raw)

## Image 1

In [ ]:
print(result_json[0]['description'])

In [ ]:
Image(filename="../../resources/generated1.png", retina=True)

## Image 2

In [ ]:
print(result_json[1]['description'])

In [ ]:
Image(filename="../../resources/generated2.png", retina=True)

## Image 3

In [ ]:
print(result_json[2]['description'])

In [ ]:
Image(filename="../../resources/generated3.png", retina=True)

# Improvements?

Ideas are welcome!

I can show you the way but you decide to walk it.